<a href="https://colab.research.google.com/github/anjalibvn/ML/blob/main/Landslide_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 59.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
#importing dependencies
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


import warnings
warnings.filterwarnings('ignore')

In [ ]:
#loading dataset to a pandaframe
data = pd.read_csv('/content/drive/MyDrive/forecast_data.csv',low_memory=False)

In [ ]:
# Many columns are same just with differnt units, for ex wind_mph and wind_kph. So, it's better to remove the redundancy
redundant_columns = ['temp_f', 'wind_kph', 'pressure_in', 'precip_in', 'feelslike_f', 'windchill_f', 'heatindex_f', 'dewpoint_f', 'vis_miles', 'chance_of_rain', 'chance_of_snow','gust_kph']

# Drop the redundant information
data.drop(redundant_columns, axis=1, inplace=True)

In [ ]:
# Encode the categorical text values
#cols_to_encode = ['condition','wind_dir', 'state', 'city']
#for col in cols_to_encode:
le = LabelEncoder()
data['condition'] = le.fit_transform(data['condition'])
data['wind_dir'] = le.fit_transform(data['wind_dir'])
data['state'] = le.fit_transform(data['state'])
data['city'] = le.fit_transform(data['city'])




In [ ]:
data.head()

,time_epoch,time,temp_c,is_day,condition,wind_mph,wind_degree,wind_dir,pressure_mb,precip_mm,...,feelslike_c,windchill_c,heatindex_c,dewpoint_c,will_it_rain,will_it_snow,vis_km,gust_mph,state,city
0,1634236200,2021-10-15 00:00,23.2,0,0,4.3,41,4,1006.0,0.0,...,25.1,23.2,25.1,16.7,0,0,10.0,8.9,0,7
1,1634239800,2021-10-15 01:00,23.0,0,0,4.1,34,4,1005.0,0.0,...,25.0,23.0,25.0,16.5,0,0,10.0,8.6,0,7
2,1634243400,2021-10-15 02:00,22.7,0,0,4.0,28,5,1005.0,0.0,...,24.9,22.7,24.9,16.4,0,0,10.0,8.4,0,7
3,1634247000,2021-10-15 03:00,22.5,0,0,3.8,22,5,1005.0,0.0,...,24.8,22.5,24.8,16.2,0,0,10.0,8.1,0,7
4,1634250600,2021-10-15 04:00,22.3,0,0,3.7,23,5,1005.0,0.0,...,24.7,22.3,24.7,16.1,0,0,10.0,7.9,0,7


In [ ]:
# drop the time column
data.drop('time', axis=1, inplace=True)


In [ ]:
X = data.drop('will_it_rain', axis=1)
y =data['will_it_rain']

X_columns = X.columns
ss = StandardScaler()
X_scaled_arr = ss.fit_transform(X)

# Create a new dataframe called X_scaled
X_scaled = pd.DataFrame(X_scaled_arr, columns=X_columns)

In [ ]:
X_scaled.head()

,time_epoch,temp_c,is_day,condition,wind_mph,wind_degree,wind_dir,pressure_mb,precip_mm,humidity,cloud,feelslike_c,windchill_c,heatindex_c,dewpoint_c,will_it_snow,vis_km,gust_mph,state,city
0,-1.723372,-0.406932,-0.955145,-1.335351,-0.273082,-1.321361,-0.682993,-0.680838,-0.249591,-0.225008,-1.098891,-0.330904,-0.400067,-0.364536,-0.341289,0.0,0.370733,0.229968,-1.690892,-1.544443
1,-1.705332,-0.448620,-0.955145,-1.335351,-0.343330,-1.388965,-0.682993,-0.971657,-0.249591,-0.225008,-1.069833,-0.347982,-0.441333,-0.382056,-0.376280,0.0,0.370733,0.157522,-1.690892,-1.544443
2,-1.687291,-0.511151,-0.955145,-1.335351,-0.378454,-1.446912,-0.478701,-0.971657,-0.249591,-0.225008,-1.011717,-0.365060,-0.503232,-0.399576,-0.393775,0.0,0.370733,0.109225,-1.690892,-1.544443
3,-1.669250,-0.552838,-0.955145,-1.335351,-0.448701,-1.504859,-0.478701,-0.971657,-0.249591,-0.225008,-0.953600,-0.382138,-0.544498,-0.417095,-0.428765,0.0,0.370733,0.036779,-1.690892,-1.544443
4,-1.651210,-0.594525,-0.955145,-1.335351,-0.483825,-1.495201,-0.478701,-0.971657,-0.249591,-0.178603,-0.953600,-0.399216,-0.585764,-0.434615,-0.446260,0.0,0.370733,-0.011518,-1.690892,-1.544443


Feature Selection

In [ ]:
# Using recursive feature elimination to select optimal number of features

# Using an ensemble model like Random Forest to use as the model.

'''
Q. How to select a model for using RFE?
A. One approach, use a model that offers scoring of features, like feature_importances_ in Random Forest.
'''

rfe_model = RandomForestClassifier()
rfe = RFE(rfe_model, verbose=3)
X_rfe = rfe.fit_transform(X_scaled, y)

Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.


In [ ]:
# Extract the important features
feature_series = pd.Series(rfe.support_, index=X_columns)
important_features = list(feature_series[feature_series == True].index)

In [ ]:
print(important_features)

['time_epoch', 'condition', 'wind_degree', 'precip_mm', 'humidity', 'cloud', 'heatindex_c', 'dewpoint_c', 'vis_km', 'gust_mph']


In [ ]:
# Prepare the final dataset
X_final = X_scaled[important_features] 
X_final =X_final.drop('precip_mm',axis=1)



Classification

In [ ]:
# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)


LogisticRegression()

In [ ]:
def rainfall_predictor(time_epoch, condition, wind_degree, precip_mm, humidity, cloud, heatindex_c, dewpoint_c, vis_km):
    inp = np.array([[time_epoch],[condition],[wind_degree],[precip_mm],[humidity],[cloud],[heatindex_c],[dewpoint_c],[vis_km]])
    inp=inp.reshape(1,-1)
    x=lr.predict(inp)
    if(x==1):
      return('RAINFALL EXPECTED')
    else:
      return('SAFE TO GO!!!')  
demo = gr.Interface(fn=rainfall_predictor,
                    inputs=["number","number","number","number","number","number","number","number","number"],
                    outputs="text")
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bfd3200e-8437-416b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
X_train_prediction = lr.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,y_train)

In [ ]:
print(training_data_accuracy)

0.8994250443899552
